# Results 5 June

In [ ]:
from tools import *
import pandas as pd
import seaborn as sns
from matplotlib import cm
from scipy.stats import ks_2samp
from IPython.display import display
style.use('ggplot')
d = manual_load_data('clean_data.pkl')
%config InlineBackend.figure_format = 'retina'

In [ ]:
markers = {}
markers['P07'] = {'index': 'index8', 'thumb': 'thumb9', 'wrist': 'wrist11'}
markers['P08'] = {'index': 'index8', 'thumb': 'thumb9', 'wrist': 'wrist11'}
markers['P09'] = {'index': 'index7', 'thumb': 'thumb9', 'wrist': 'wrist11'}
markers['P11'] = {'index': 'index8', 'thumb': 'thumb9', 'wrist': 'wrist11'}
markers['P12'] = {'index': 'index8', 'thumb': 'thumb9', 'wrist': 'wrist11'}
markers['P16'] = {'index': 'index8', 'thumb': 'thumb9', 'wrist': 'wrist12'}
markers['P18'] = {'index': 'index8', 'thumb': 'thumb9', 'wrist': 'wrist11'}
markers['P19'] = {'index': 'index8', 'thumb': 'thumb9', 'wrist': 'wrist11'}

pd.DataFrame(markers)

## Distance to object's centre for index & gaze

In [ ]:
long_table = pd.DataFrame(columns = ['p', 'condition', 'trial', 'indx-obj', 'gaze-obj'])

for pid, subj in d.items():
    indx = markers[pid]['index'] + 'x'
    for tid, trial in subj['trials'].items():
        
        x1 = trial[indx][-1] - trial['objectx'][-1]
        x2 = trial['averagexeye'][-1] - trial['objectx'][-1]
        
        if 'RightToLeft' in trial['name']:
            x1 *= -1
            x2 *= -1
            
        cond = trial['name'][6:-4]
        
        to_add = pd.Series([pid, cond, tid, x1, x2], long_table.columns.values.tolist())
        long_table = long_table.append(to_add, ignore_index = True)

In [ ]:
long_table[:20]

In [ ]:
inter1 = long_table.groupby(['participant', 'condition']).aggregate(np.mean)
inter2 = inter1.reset_index(level = ['condition'])
meanPerCondition = inter2.groupby('condition').aggregate(np.mean)

meanPerCondition

In [ ]:
inter3 = inter1.reset_index(level = ['participant', 'condition'])
anovaTable = inter3.pivot(index = 'participant', columns = 'condition', values = 'indx-obj')

### Accuracy per condition

In [ ]:
anovaTable.mean(axis = 0).plot(kind = 'bar', yerr = anovaTable.std(axis = 0) / np.sqrt(len(anovaTable)))
plt.show()

### Accuracy per participant

In [ ]:
anovaTable.mean(axis = 1).plot(kind = 'bar', yerr = anovaTable.std(axis = 1) / np.sqrt(len(anovaTable.columns)))
plt.show()

## Fixations

In [ ]:
cues = [0.353 + n * 0.072 for n in range(8)]
print cues

In [ ]:
# Create a common fixations table

fixations = pd.DataFrame()

for pid, subj in d.items():
    
    index_x = markers[pid]['index'] + 'x'

    for tid, trial in subj['trials'].items():
        
        to_add = pd.DataFrame(trial['fix'])
        
        # identify the reach onset frame
        sm = trial['startmovement']
        ro_frame = np.where(np.array(sm) == 1)[0][0]
                
        # find the eye and index to object distances
        index_object = trial[index_x][-1] - trial['objectx'][-1]
        eyes_object_final = to_add.iloc[-1]['centre_x'] - trial['objectx'][-1] # final fixation in a trial
        
        ro_fix = to_add[(to_add['start_frame'] <= ro_frame) & (to_add['end_frame'] >= ro_frame)]
        if not ro_fix['centre_x'].any():
            ro_fix = to_add[to_add['start_frame'] >= ro_frame].iloc[0]
            
        eyes_object_ro = ro_fix['centre_x'] - trial['objectx'][ro_frame]
        
        if 'RightToLeft' in trial['name']:
            index_object *= -1
            eyes_object_final *= -1
            eyes_object_ro *= -1
        
        # identify the last fixation in a trial (or just the last row)
        to_add['time_interval'] = None
        to_add.loc[to_add.index[-1], 'time_interval'] = 3
        
        # add the data
        to_add['p'] = pid
        to_add['condition'] = trial['name'][6:-4]
        to_add['trial'] = tid
        to_add['ro_frame'] = ro_frame
        to_add['index_object'] = index_object
        to_add['eyes_object_final'] = eyes_object_final
        to_add['eyes_object_ro'] = eyes_object_ro
        
        # remove the dispersion column
        to_add.drop('dispersion', axis = 1, inplace = True)
 
        fixations = fixations.append(to_add, ignore_index = True)
        
# fixations = fixations[['p', 'condition', 'trial', 'centre_x', 'centre_z', 'duration', 'start_frame', 'end_frame',
#                        'ro_frame', 'time_interval']]


In [ ]:
fixations[:10]

In [ ]:
# Determine the time interval 
# 0 = before occlusion. frames 0-495
# 1 = visual pursuit. frames 496 - reach onset
# 2 = reach onset - time of contact-1
# 3 = time of contact

time_interval = []

for index, row in fixations.iterrows():
    if row['time_interval'] == 3:
        time_interval.append(3)
        continue
    elif row['start_frame'] <= 495:
        time_interval.append(0)
    elif 495 < row['start_frame'] < row['ro_frame']:
        time_interval.append(1)
    elif row['start_frame'] >= row['ro_frame']:
        time_interval.append(2)

fixations['time_interval'] = time_interval

In [ ]:
# Detect the off-screen outliers

fixations['off_screen'] = (fixations.centre_x < 0.34) | (fixations.centre_x > 0.87) | \
    (fixations.centre_z < 0.18) | (fixations.centre_z > 0.48)
    
fixations['off_screen'].value_counts()

In [ ]:
# Find the distances between fixations and the closest cue

closest_cue = []
distance = []

for index, row in fixations.iterrows():
    dist = [row['centre_x'] - cue for cue in cues]
    ind, = np.where(np.abs(dist) == min(np.abs(dist)))
    closest_cue.append(ind[0])
    distance.append(dist[ind[0]])
    
fixations['closest_cue'] = closest_cue
fixations['distance_to'] = distance

fixations[:5]

In [ ]:
# Number of fixations per closest cue

fixations['closest_cue'].value_counts(sort = False).plot(kind = 'bar')
plt.show()

In [ ]:
# Number of fixations in the time intervals

fixations['time_interval'].value_counts(sort = False).plot(kind = 'bar')
plt.show()

### Fixations' location during visual pursuit

In [ ]:
# Fixations for occlusion, cue vs no cue, left to right


ocl_fixations = fixations[(fixations['condition'] == 'Occlusion_Cue_LeftToRight') & \
                          (fixations['off_screen'] == False) & \
                          (fixations['time_interval'] == 1)]


onl_fixations = fixations[(fixations['condition'] == 'Occlusion_NoCue_LeftToRight') & \
                          (fixations['off_screen'] == False) & \
                          (fixations['time_interval'] == 1)]


In [ ]:
fig = plt.figure(figsize = [15, 10])
ax1 = fig.add_subplot(211)
draw_cues(ax1, ybottom = 0.18, ytop = 0.48)
ax1.plot(ocl_fixations['centre_x'], ocl_fixations['centre_z'], 'b.', alpha = 0.3, markersize = 10)
ax1.set_title('cues')

ax2 = fig.add_subplot(212)
draw_cues(ax2, ybottom = 0.18, ytop = 0.48)
ax2.plot(onl_fixations['centre_x'], onl_fixations['centre_z'], 'b.', alpha = 0.3, markersize = 10)
ax2.set_title('no cues')

plt.show()


In [ ]:
fig = plt.figure(figsize = [10, 10])
ax1 = fig.add_subplot(211)
ax1.hist(ocl_fixations['centre_x'], bins = 50)
draw_cues(ax1)

ax2 = fig.add_subplot(212, sharex = ax1)
ax2.hist(onl_fixations['centre_x'], bins = 50)
draw_cues(ax2)

plt.show()

In [ ]:
# Overlay the cues & plot the closest fixations. 0 = cue position

x1 = ocl_fixations['distance_to']
x2 = onl_fixations['distance_to']

fig = plt.figure(figsize = [10, 10])
ax1 = fig.add_subplot(211)
ax1.hist(x1, bins = 30)
ax2 = fig.add_subplot(212, sharey = ax1)
ax2.hist(x2, bins = 30)
plt.show()

In [ ]:

for subj in d.values():
    for trial in subj['trials'].values():
        if 'Occlusion_Cue_LeftToRight' in trial['name']:
            plt.plot(trial['fix']['centre_x'], trial['fix']['centre_z'], 'b-', alpha = 0.1, linewidth = 1)
            plt.plot(trial['fix']['centre_x'][-1], trial['fix']['centre_z'][-1], 'r.')

draw_cues(plt.gca(), ybottom = 0.18, ytop = 0.48)
plt.ylim(0.18, 0.47)
plt.show()

In [ ]:
# Kolmogorov-Smirnov two-sample test for ocl vs onl

res = ks_2samp(x1, x2)
print 'ks = {}\np = {}'.format(round(res[0], 4), round(res[1], 4))

### Fixations' location during reach

In [ ]:
ocl_fix_reach = fixations[(fixations['condition'] == 'Occlusion_Cue_LeftToRight') & \
                          (fixations['off_screen'] == False) & \
                          (fixations['time_interval'] == 2)]

ocl_fix_final = fixations[(fixations['condition'] == 'Occlusion_Cue_LeftToRight') & \
                          (fixations['off_screen'] == False) & \
                          (fixations['time_interval'] == 3)]


onl_fix_reach = fixations[(fixations['condition'] == 'Occlusion_NoCue_LeftToRight') & \
                          (fixations['off_screen'] == False) & \
                          (fixations['time_interval'] == 2)]

onl_fix_final = fixations[(fixations['condition'] == 'Occlusion_NoCue_LeftToRight') & \
                          (fixations['off_screen'] == False) & \
                          (fixations['time_interval'] == 3)]

In [ ]:
# Fixations after the reach onset

fig = plt.figure(figsize = [15, 10])

ax1 = fig.add_subplot(211)
ax1.plot(ocl_fix_reach['centre_x'], ocl_fix_reach['centre_z'], 'b.', alpha = 0.5)
draw_cues(ax1, ybottom = 0.18, ytop = 0.48)
ax1.set_xlim(0.34, 0.87)
ax1.set_ylim(0.18, 0.48)

ax2 = fig.add_subplot(212, sharex = ax1, sharey = ax1)
ax2.plot(onl_fix_reach['centre_x'], onl_fix_reach['centre_z'], 'b.', alpha = 0.5)
draw_cues(ax2, ybottom = 0.18, ytop = 0.48)

plt.show()

In [ ]:
# Distribution relative to the cue

x1 = ocl_fix_reach['distance_to']
x2 = onl_fix_reach['distance_to']

fig = plt.figure(figsize = [10, 10])
ax1 = fig.add_subplot(211)
ax1.hist(x1, bins = 10)
ax2 = fig.add_subplot(212, sharey = ax1)
ax2.hist(x2, bins = 10)
plt.show()

### -------------------- Fixations' duration --------------------

In [ ]:
from matplotlib import cm

fig = plt.figure(figsize = [15, 10])
ax1 = fig.add_subplot(211)
draw_cues(ax1, ybottom = 0.18, ytop = 0.48)
ax1.scatter(ocl_fixations['centre_x'], ocl_fixations['centre_z'], c = ocl_fixations['duration'],
            alpha = 0.5, cmap = cm.viridis_r, s = 20)
ax1.set_title('cues')
# ax1.set_xlim(0.5, 0.8)
# ax1.set_ylim(0.3, 0.4)

ax2 = fig.add_subplot(212)
draw_cues(ax2, ybottom = 0.18, ytop = 0.48)
ax2.scatter(onl_fixations['centre_x'], onl_fixations['centre_z'], c = onl_fixations['duration'],
            alpha = 0.5, cmap = cm.viridis_r, s = 20)
ax2.set_title('no cues')

plt.show()

In [ ]:
# Linear model duration ~ distance for ocl

from scipy.stats import linregress

slope, intercept, r, p, stderr = linregress(abs(ocl_fixations['distance_to']), ocl_fixations['duration'])
print 'r = {}\np = {}'.format(round(r, 3), round(p, 3))
y = slope * abs(ocl_fixations['distance_to']) + intercept

plt.plot(abs(ocl_fixations['distance_to']), ocl_fixations['duration'], '.')
plt.plot(abs(ocl_fixations['distance_to']), y, 'b-', linewidth = 0.5)
plt.xlabel('distance from cue (m)')
plt.ylabel('duration (s)')
plt.show()

In [ ]:
# Distribution of fixations' duration

plt.hist(ocl_fixations['duration'], bins = 30)
plt.xlabel('ocl fixations duration (s)')
plt.show()

In [ ]:
# Fixation duration against location on the x-axis

fig = plt.figure(figsize = [15, 10])

ax1 = fig.add_subplot(211)
ax1.plot(ocl_fixations['centre_x'], ocl_fixations['duration'], '.', markersize = 10, alpha = 0.5)
draw_cues(ax1, ybottom = 0, ytop = 1.4)
ax1.set_ylabel('duration (s)')

ax2 = fig.add_subplot(212, sharex = ax1, sharey = ax1)
ax2.plot(onl_fixations['centre_x'], onl_fixations['duration'], '.', markersize = 10, alpha = 0.5)
draw_cues(ax2, ybottom = 0, ytop = 1.4)
ax2.set_xlabel('x location (m)')
ax2.set_ylabel('duration (s)')

plt.show()

In [ ]:
fig = plt.figure(figsize = [15, 10])

ax1 = fig.add_subplot(211)
ax1.scatter(ocl_fixations['centre_x'], ocl_fixations['duration'], c = ocl_fixations['start_frame'], cmap = cm.viridis)
draw_cues(ax1, ybottom = 0, ytop = 1.4)
ax1.set_ylabel('duration (s)')

ax2 = fig.add_subplot(212, sharex = ax1, sharey = ax1)
ax2.scatter(onl_fixations['centre_x'], onl_fixations['duration'], c = onl_fixations['start_frame'], cmap = cm.viridis)
draw_cues(ax2, ybottom = 0, ytop = 1.4)
ax2.set_xlabel('x location (m)')
ax2.set_ylabel('duration (s)')

plt.show()

In [ ]:
plt.plot(ocl_fixations['start_frame'], ocl_fixations['duration'], '.')
plt.show()

## Reaching movement

In [ ]:
def auc(x, y, z):
    vel = np.sqrt(np.diff(x)**2 + np.diff(y)**2 + np.diff(z)**2) # in m/frame. divide by sampling freq to get m/s
    d = sum(vel)
    return d

In [ ]:
# Find the reach travel distance for wrist

td = pd.DataFrame(columns = ['participant', 'condition', 'travel_distance'])

for pid, subj in d.items():
    for trial in subj['trials'].values():
        
        x = trial[markers[pid]['wrist'] + 'x']
        y = trial[markers[pid]['wrist'] + 'y']
        z = trial[markers[pid]['wrist'] + 'z']
        
        dist = auc(x, y, z)
        cond = trial['name'][6:-4]
        
        to_add = pd.Series([pid, cond, dist], td.columns.values.tolist())
        td = td.append(to_add, ignore_index = True)
        
td[:5]

In [ ]:
a = td.groupby(['participant', 'condition']).aggregate(np.mean).reset_index()
b = a.pivot(index = 'participant', columns = 'condition', values = 'travel_distance')
c = b.mean(axis = 0)
# b.loc[:, ['Occlusion_Cue_LeftToRight', 'Visible_Cue_LeftToRight']].plot(kind = 'bar')
c.plot(kind = 'bar',  yerr = b.std(axis = 0) / np.sqrt(len(b)))
plt.ylabel('travel distance (m)')
plt.show()

In [ ]:
# Reach trajectories on x-y axes for wrist

fig = plt.figure(figsize = [15, 10])
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

for pid, subj in d.items():
    for trial in subj['trials'].values():
        x = trial[markers[pid]['wrist'] + 'x']
        y = trial[markers[pid]['wrist'] + 'y']
        if 'Occlusion_Cue_LeftToRight' in trial['name'] or 'Occlusion_NoCue_LeftToRight' in trial['name']:
            ax1.plot(x, y, color = 'b', alpha = 0.1)
        elif 'Occlusion_Cue_RightToLeft' in trial['name'] or 'Occlusion_NoCue_RightToLeft' in trial['name']:
            ax1.plot(x, y, color = 'r', alpha = 0.1)
        elif 'Visible_Cue_LeftToRight' in trial['name'] or 'Visible_NoCue_LeftToRight' in trial['name']:
            ax2.plot(x, y, color = 'b', alpha = 0.1)
        elif 'Visible_Cue_RightToLeft' in trial['name'] or 'Visible_NoCue_RightToLeft' in trial['name']:
            ax2.plot(x, y, color = 'r', alpha = 0.1)

ax1.set_ylim(0, 0.5)
ax2.set_ylim(0, 0.5)
ax1.set_xlim(0.34, 0.87)
ax2.set_xlim(0.34, 0.87)
plt.show()

In [ ]:
# Reach trajectories on x-y axes for index

fig = plt.figure(figsize = [15, 10])
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

for pid, subj in d.items():
    for trial in subj['trials'].values():
        x = trial[markers[pid]['index'] + 'x']
        y = trial[markers[pid]['index'] + 'y']
        if 'Occlusion_Cue_LeftToRight' in trial['name'] or 'Occlusion_NoCue_LeftToRight' in trial['name']:
            ax1.plot(x, y, color = 'b', alpha = 0.1)
        elif 'Occlusion_Cue_RightToLeft' in trial['name'] or 'Occlusion_NoCue_RightToLeft' in trial['name']:
            ax1.plot(x, y, color = 'r', alpha = 0.1)
        elif 'Visible_Cue_LeftToRight' in trial['name'] or 'Visible_NoCue_LeftToRight' in trial['name']:
            ax2.plot(x, y, color = 'b', alpha = 0.1)
        elif 'Visible_Cue_RightToLeft' in trial['name'] or 'Visible_NoCue_RightToLeft' in trial['name']:
            ax2.plot(x, y, color = 'r', alpha = 0.1)

ax1.set_ylim(0, 0.56)
ax2.set_ylim(0, 0.56)
ax1.set_xlim(0.34, 0.87)
ax2.set_xlim(0.34, 0.87)
plt.show()

In [ ]:
# Get the index accuracy and mean distance of fixations table

mean_fixations = fixations.groupby(['p', 'trial'])['distance_to'] \
                          .apply(lambda x: np.mean(np.abs(x))) \
                          .reset_index()

In [ ]:
common = pd.merge(mean_fixations, long_table, on = ['p', 'trial'])
display(common[:10])

ocl = common[common['condition'] == 'Occlusion_Cue_LeftToRight']
onl = common[common['condition'] == 'Occlusion_NoCue_LeftToRight']

In [ ]:
slope, intercept, r, p, stderr = linregress(ocl['distance_to'], ocl['indx-obj'])
print 'r = {}\np = {}'.format(round(r, 2), round(p, 3))

y = intercept + slope * ocl['distance_to']

plt.plot(ocl['distance_to'], ocl['indx-obj'], '.')
plt.plot(ocl['distance_to'], y, 'b-', linewidth = 0.5)
plt.show()